# Setup

In [1]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !apt-get install lz4

In [ ]:
# !pip install pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg

In [ ]:
# !pip install dcm2niix pyaml idc-index pyradiomics SimpleITK==2.3.0  pyplastimatch==0.4.2 highdicom==0.21.0 pydicom==2.4.3
# from pyplastimatch.utils.install import install_precompiled_binaries
# install_precompiled_binaries()

In [ ]:
# dcmqi_release_url = "https://github.com/QIICR/dcmqi/releases/download/v1.3.4/dcmqi-1.3.4-linux.tar.gz"
# dcmqi_download_path = "dcmqi-1.3.4-linux.tar.gz"
# dcmqi_path = "dcmqi-1.3.4-linux"
# !wget -O $dcmqi_download_path $dcmqi_release_url\
# && tar -xvf $dcmqi_download_path\
# && mv $dcmqi_path/bin/* /bin\
# && rm -r $dcmqi_download_path $dcmqi_path

In [6]:
# Imports

import os
import json
import glob

import numpy as np

import shutil

import pyplastimatch as pypla

import math
import pandas as pd
import SimpleITK as sitk

from os.path import abspath

import pydicom

from idc_index import index
client = index.IDCClient()

# Global Variables


In [9]:
#absolute path output
OUTPUT_PATH = ""#"TEST"#"radiomics_task"#""
#input MHUB zip file
SEG_OUTPUT = ""#"ai_combination_archive.tar.lz4"#"/content/radiomics_idc_output.tar.lz4"#""
#segments parameters
dicom_sr_CodeValues_lst = []#[ "41216001", "279706003", "399384005" ]
dicom_sr_codeMeaning_lst = []#[ "Prostatic_structure", "Structure_of_peripheral_glandular_zone_of_prostate_(body_structure)", "Structure_of_transition_zone_of_prostate_(body_structure)" ]
dicom_sr_CodingSchemeDesignator_lst = []#[ "SCT", "SCT", "SCT" ]
#AlgorithmName
terraRadSeriesDescription = ""#"bamf_nnunet_mr_prostate"

In [10]:
SEG_OUTPUT_UNZIP = os.path.join(OUTPUT_PATH, "seg_unzip")

seg_base_folder = os.path.join(OUTPUT_PATH, "seg_objects")

seg_nii_folder = os.path.join(seg_base_folder, "nii")
seg_nii_resampled_folder = os.path.join(seg_base_folder, "nii_resampled")
seg_nii_resampled_renamed_folder = os.path.join(seg_base_folder, "nii_resampled_renamed")
seg_sr_dicom_folder = os.path.join(seg_base_folder, "dicom_sr")
seg_sr_json_dicom_folder = os.path.join(seg_base_folder, "dicom_sr_json")

#IDC EVALUATION DATA
## Image
IDC_IMAGE_DICOM = os.path.join(OUTPUT_PATH, "idc_image_data/dicom")
IDC_IMAGE_NII = os.path.join(OUTPUT_PATH, "idc_image_data/nii")
IDC_IMAGE_NRRD = os.path.join(OUTPUT_PATH,"idc_image_data/nrrd")

In [11]:
def setup_folders():
	"""
	creates environment data and results folder, first deletes current ones and then creates new ones
	"""
	for folder_setup in [OUTPUT_PATH, SEG_OUTPUT_UNZIP, seg_base_folder,
											 seg_nii_folder, seg_nii_resampled_folder,
											 seg_sr_dicom_folder, seg_sr_json_dicom_folder,
											 IDC_IMAGE_DICOM, IDC_IMAGE_NII, IDC_IMAGE_NRRD]:
		!rm -rf $folder_setup
		!mkdir -p $folder_setup

In [12]:
setup_folders()

# Utility functions

## Conversion DICOM <---> NII

In [37]:
def convert_image_dcm_to_nii(input_path, output_path_root, prefix="", format="nii"):
	"""
	Conversion of DICOM MR data to NIFTI using dcm2niix
	input_path : str, folder containing DICOM instances .dcm
	output_path_root : str, output folder
	prefix : str, prefix for output file name
	"""
	if not os.path.exists(output_path_root):
		!mkdir -p $output_path_root
	!dcm2niix -z y -m y -f %i_{prefix} -o $output_path_root $input_path

## Resampling

In [38]:
def resample_preds(input_path_nnunet_preds="", input_path_t2_idc="", output_path=""):
	"""
	Resampling of nnunet_preds to reference MR T2 volumes used as input image
	input_path_nnunet_preds : str, folder containing nnunet preds
	input_path_t2_idc : str, folder containing NIFTI IDC MR T2 volumes
	output_path : str, folder output
	"""
	for pred_path in sorted(glob.glob(os.path.join(input_path_nnunet_preds, "*.nii.gz"))):
		print(f"pred path : {pred_path}")
		resample_args_to_t2_origin = {"input" : pred_path,#change to pred path if no largest_component_retrieval necessary
													"output" : os.path.join(output_path,
																									f"{pred_path.split('/')[-1][:-7]}.nii.gz"),
													"fixed" : input_path_t2_idc,
													"interpolation" : "nn"}

		pypla.resample(verbose = False, **resample_args_to_t2_origin)
		print()

## Combine segs

In [39]:
def add_segments_json(input_json, segmentAttributesProstate_lst):
	assert os.path.exists(input_json)
	inp_json = json.load(open(input_json)).copy()
	print(inp_json["segmentAttributes"])
	print(type(inp_json["segmentAttributes"]))
	temp_lst = inp_json["segmentAttributes"].copy()
	temp_lst.append(segmentAttributesProstate_lst)
	inp_json["segmentAttributes"] = temp_lst
	print(inp_json["segmentAttributes"])

	return inp_json

In [40]:
def retrieve_meta_df_from_json(meta_json_path):
	out_segment_list = []
	json_dic = json.load(open(meta_json_path))
	for segment_iter in json_dic["segmentAttributes"]:
		temp_dic = {}
		temp_dic = segment_iter
		out_segment_list.append(temp_dic[0])
	out_df = pd.DataFrame.from_dict(out_segment_list)
	df_SegmentedPropertyTypeCodeSequence = pd.json_normalize(out_df['SegmentedPropertyTypeCodeSequence']).rename(columns={'CodeMeaning': 'FindingSite_CodeMeaning',
																																																												'CodeValue':'FindingSite_CodeValue',
																																																												'CodingSchemeDesignator': 'FindingSite_CodingSchemeDesignator'})
	df_SegmentedPropertyCategoryCodeSequence = pd.json_normalize(out_df['SegmentedPropertyCategoryCodeSequence']).rename(columns={'CodeMeaning': 'Finding_CodeMeaning',
																																																												'CodeValue':'Finding_CodeValue',
																																																												'CodingSchemeDesignator': 'Finding_CodingSchemeDesignator'})
	df_columns_lst = list(out_df.columns.values)
	cols_to_retrieve = []
	for col_test in ["SegmentAlgorithmName","SegmentAlgorithmType",
									 "SegmentDescription","SegmentLabel"]:
		if col_test in df_columns_lst:
				cols_to_retrieve.append(col_test)
		else:
			pass
	df_final = df_SegmentedPropertyTypeCodeSequence.join(df_SegmentedPropertyCategoryCodeSequence).join(out_df[["labelID"]\
		+ cols_to_retrieve])
	df_final["segment"] = df_final["FindingSite_CodeMeaning"]
	df_final["SeriesDescription"] = json_dic["SeriesDescription"]
	df_final["SeriesNumber"] = json_dic["SeriesNumber"]
	df_final["BodyPartExamined"] = json_dic["BodyPartExamined"]
	df_final = df_final.reset_index().drop(columns=["index"])
	return df_final

In [41]:
def generateProstateDcmqiMetadata(labelID, SegmentAlgorithmType,
																	SegmentAlgorithmName,
																	SegmentLabel,
																	SegmentDescription,
																	SegmentedPropertyCategoryCodeSequenceCodeValue,
																	SegmentedPropertyCategoryCodeSequenceCodingSchemeDesignator,
																	SegmentedPropertyCategoryCodeSequenceCodeMeaning,
																	SegmentedPropertyTypeCodeSequenceCodeValue,
																	SegmentedPropertyTypeCodeSequenceCodingSchemeDesignator,
																	SegmentedPropertyTypeCodeSequenceCodeMeaning,
																	recommendedDisplayRGBValue):
	out_lst = []
	if SegmentAlgorithmName is not None:
		out_lst.append({"SegmentAlgorithmName" : SegmentAlgorithmName,
										"SegmentAlgorithmType" : SegmentAlgorithmType,
										"SegmentLabel" : SegmentLabel,
										"SegmentDescription" : SegmentDescription,
										"SegmentedPropertyCategoryCodeSequence" : {
											"CodeMeaning" : SegmentedPropertyCategoryCodeSequenceCodeMeaning,
											"CodeValue" : SegmentedPropertyCategoryCodeSequenceCodeValue,
											"CodingSchemeDesignator" : SegmentedPropertyCategoryCodeSequenceCodingSchemeDesignator,
										},
										"SegmentedPropertyTypeCodeSequence" : {
												"CodeMeaning" : SegmentedPropertyTypeCodeSequenceCodeMeaning,
												"CodeValue" : SegmentedPropertyTypeCodeSequenceCodeValue,
												"CodingSchemeDesignator" : SegmentedPropertyTypeCodeSequenceCodingSchemeDesignator,
										},
										"labelID" : int(labelID),
										"recommendedDisplayRGBValue" : [int(x) for x in recommendedDisplayRGBValue]})
	else:
		out_lst.append({"SegmentAlgorithmType" : SegmentAlgorithmType,
										"SegmentLabel" : SegmentLabel,
										"SegmentDescription" : SegmentDescription,
										"SegmentedPropertyCategoryCodeSequence" : {
											"CodeMeaning" : SegmentedPropertyCategoryCodeSequenceCodeMeaning,
											"CodeValue" : SegmentedPropertyCategoryCodeSequenceCodeValue,
											"CodingSchemeDesignator" : SegmentedPropertyCategoryCodeSequenceCodingSchemeDesignator,
										},
										"SegmentedPropertyTypeCodeSequence" : {
												"CodeMeaning" : SegmentedPropertyTypeCodeSequenceCodeMeaning,
												"CodeValue" : SegmentedPropertyTypeCodeSequenceCodeValue,
												"CodingSchemeDesignator" : SegmentedPropertyTypeCodeSequenceCodingSchemeDesignator,
										},
										"labelID" : int(labelID),
										"recommendedDisplayRGBValue" : [int(x) for x in recommendedDisplayRGBValue]})
	return out_lst

In [42]:
def combine_ind_seg(input_nii_folder, output_file_path):
	assert os.path.exists(input_nii_folder)
	out_arr = np.zeros_like(sitk.GetArrayFromImage(sitk.ReadImage(glob.glob(os.path.join(input_nii_folder, "*.nii.gz"))[0])))
	for segment_processed in sorted(glob.glob(os.path.join(input_nii_folder, "*.nii.gz"))):
		out_arr += sitk.GetArrayFromImage(sitk.ReadImage(segment_processed))
	res_img = sitk.GetImageFromArray(out_arr)
	res_img.CopyInformation(sitk.ReadImage(glob.glob(os.path.join(input_nii_folder, "*.nii.gz"))[0]))
	# Save output segment
	sitk.WriteImage(res_img, output_file_path)

## DICOM metadata

In [43]:
def retrieve_image_serieUID_dcm_seg(inp_dicom_seg):
	return str(pydicom.dcmread(inp_dicom_seg).ReferencedSeriesSequence[0].SeriesInstanceUID)

In [44]:
def get_tag_pydicom(path_dcm_file : str, tag_name : str):
	dataset = pydicom.dcmread(path_dcm_file)
	return dataset[tag_name]

In [45]:
def retrieve_meta_df_from_json(meta_json_path):
	out_segment_list = []
	json_dic = json.load(open(meta_json_path))
	for segment_iter in json_dic["segmentAttributes"]:
		temp_dic = {}
		temp_dic = segment_iter
		out_segment_list.append(temp_dic[0])
	out_df = pd.DataFrame.from_dict(out_segment_list)
	df_SegmentedPropertyTypeCodeSequence = pd.json_normalize(out_df['SegmentedPropertyTypeCodeSequence']).rename(columns={'CodeMeaning': 'FindingSite_CodeMeaning',
																																																												'CodeValue':'FindingSite_CodeValue',
																																																												'CodingSchemeDesignator': 'FindingSite_CodingSchemeDesignator'})
	df_SegmentedPropertyCategoryCodeSequence = pd.json_normalize(out_df['SegmentedPropertyCategoryCodeSequence']).rename(columns={'CodeMeaning': 'Finding_CodeMeaning',
																																																												'CodeValue':'Finding_CodeValue',
																																																												'CodingSchemeDesignator': 'Finding_CodingSchemeDesignator'})
	df_columns_lst = list(out_df.columns.values)
	cols_to_retrieve = []
	for col_test in ["SegmentAlgorithmName","SegmentAlgorithmType",
									 "SegmentDescription","SegmentLabel"]:
		if col_test in df_columns_lst:
				cols_to_retrieve.append(col_test)
		else:
			pass
	df_final = df_SegmentedPropertyTypeCodeSequence.join(df_SegmentedPropertyCategoryCodeSequence).join(out_df[["labelID"]\
		+ cols_to_retrieve])
	df_final["segment"] = df_final["FindingSite_CodeMeaning"]
	df_final["SeriesDescription"] = json_dic["SeriesDescription"]
	df_final["SeriesNumber"] = json_dic["SeriesNumber"]
	df_final["BodyPartExamined"] = json_dic["BodyPartExamined"]
	df_final = df_final.reset_index().drop(columns=["index"])
	return df_final

In [46]:
def generateProstateDcmqiMetadata(labelID, SegmentAlgorithmType,
																	SegmentAlgorithmName,
																	SegmentLabel,
																	SegmentDescription,
																	SegmentedPropertyCategoryCodeSequenceCodeValue,
																	SegmentedPropertyCategoryCodeSequenceCodingSchemeDesignator,
																	SegmentedPropertyCategoryCodeSequenceCodeMeaning,
																	SegmentedPropertyTypeCodeSequenceCodeValue,
																	SegmentedPropertyTypeCodeSequenceCodingSchemeDesignator,
																	SegmentedPropertyTypeCodeSequenceCodeMeaning,
																	recommendedDisplayRGBValue):
	out_lst = []
	if SegmentAlgorithmName is not None:
		out_lst.append({"SegmentAlgorithmName" : SegmentAlgorithmName,
										"SegmentAlgorithmType" : SegmentAlgorithmType,
										"SegmentLabel" : SegmentLabel,
										"SegmentDescription" : SegmentDescription,
										"SegmentedPropertyCategoryCodeSequence" : {
											"CodeMeaning" : SegmentedPropertyCategoryCodeSequenceCodeMeaning,
											"CodeValue" : SegmentedPropertyCategoryCodeSequenceCodeValue,
											"CodingSchemeDesignator" : SegmentedPropertyCategoryCodeSequenceCodingSchemeDesignator,
										},
										"SegmentedPropertyTypeCodeSequence" : {
												"CodeMeaning" : SegmentedPropertyTypeCodeSequenceCodeMeaning,
												"CodeValue" : SegmentedPropertyTypeCodeSequenceCodeValue,
												"CodingSchemeDesignator" : SegmentedPropertyTypeCodeSequenceCodingSchemeDesignator,
										},
										"labelID" : int(labelID),
										"recommendedDisplayRGBValue" : [int(x) for x in recommendedDisplayRGBValue]})
	else:
		out_lst.append({"SegmentAlgorithmType" : SegmentAlgorithmType,
										"SegmentLabel" : SegmentLabel,
										"SegmentDescription" : SegmentDescription,
										"SegmentedPropertyCategoryCodeSequence" : {
											"CodeMeaning" : SegmentedPropertyCategoryCodeSequenceCodeMeaning,
											"CodeValue" : SegmentedPropertyCategoryCodeSequenceCodeValue,
											"CodingSchemeDesignator" : SegmentedPropertyCategoryCodeSequenceCodingSchemeDesignator,
										},
										"SegmentedPropertyTypeCodeSequence" : {
												"CodeMeaning" : SegmentedPropertyTypeCodeSequenceCodeMeaning,
												"CodeValue" : SegmentedPropertyTypeCodeSequenceCodeValue,
												"CodingSchemeDesignator" : SegmentedPropertyTypeCodeSequenceCodingSchemeDesignator,
										},
										"labelID" : int(labelID),
										"recommendedDisplayRGBValue" : [int(x) for x in recommendedDisplayRGBValue]})
	return out_lst

In [47]:
def add_segments_json(input_json, segmentAttributesProstate_lst):
	assert os.path.exists(input_json)
	inp_json = json.load(open(input_json)).copy()
	print(inp_json["segmentAttributes"])
	print(type(inp_json["segmentAttributes"]))
	temp_lst = inp_json["segmentAttributes"].copy()
	temp_lst.append(segmentAttributesProstate_lst)
	inp_json["segmentAttributes"] = temp_lst
	print(inp_json["segmentAttributes"])

	return inp_json

## SR reports creation

In [48]:
def get_label_and_names_from_metadata_json(dicomseg_json):

	"""Returns two lists containing the label values and the corresponding
		 CodeMeaning values

	Inputs:
		dicomseg_json : metajson file

	Outputs:
		label_values  : label values from the metajson file
		label_names   : the corresponding CodeMeaning values

	"""

	f = open(dicomseg_json)
	meta_json = json.load(f)

	print(meta_json)

	num_regions = len(meta_json['segmentAttributes'][0])
	print ('num_regions: ' + str(num_regions))

	label_values = []
	label_names = []
	for n in range(0,num_regions):
		# label_values.append(n)
		label_value = meta_json['segmentAttributes'][0][n]['labelID']
		label_name = meta_json['segmentAttributes'][0][n]['SegmentedPropertyTypeCodeSequence']['CodeMeaning']
		label_values.append(label_value)
		label_names.append(label_name)

	return label_values, label_names

In [49]:
def split_nii(input_file, output_directory, label_names):

	"""Function to split a single multilabel nii into individual nii files. Used
		 for pyradiomics feature extraction.

	Inputs:
		input_file       : input multi-label nii file
		output_directory : where to save the individual nii segments
		label_names      : the names of the labels that correspond to the order of
											 the values in the nii input_file

	Outputs:
		saves the individual nii files to the output_directory

	"""

	if not os.path.isdir(output_directory):
		os.mkdir(output_directory)

	# save with the values in the files
	nii = nib.load(input_file)
	header = nii.header
	img = nii.get_fdata()
	unique_labels = list(np.unique(img))
	unique_labels.remove(0) # remove the background

	# split and save
	num_labels = len(unique_labels)
	for n in range(0,num_labels):
		ind = np.where(img==unique_labels[n])
		vol = np.zeros((img.shape))
		vol[ind] = 1
		new_img = nib.Nifti1Image(vol, nii.affine, nii.header)
		output_filename = os.path.join(output_directory, label_names[n] + '.nii.gz')
		nib.save(new_img, output_filename)

	return

In [50]:
def compute_pyradiomics_3D_features(ct_nifti_path,
																		label_values,
																		label_names,
																		split_pred_nifti_path,
																		nnunet_shape_features_code_mapping_df):

	"""Function to compute pyradiomics 3D features for each label in a nifti file.

	Inputs:
		ct_nifti_path            : the CT nifti file
		label_values             : the label value for each of the segments from the json file
		label_names              : the corresponding label name for each of the segments
		split_pred_nifti_path    : where to save the individual nii segments needed
															 for pyradiomics
		nnunet_shape_features_code_mapping_df : the df where we will obtain the
																						list of the shape features to
																						compute

	Outputs:
		Writes the features_csv_path_nnunet to disk.
	"""

	# Get the names of the features from the nnunet_shape_features_code_mapping_df
	shape_features = list(nnunet_shape_features_code_mapping_df['shape_feature'].values)



	# Calculate features for each label and create a dataframe
	# num_labels = len(label_names)#len([f for f in os.listdir(split_pred_nifti_path) if f.endswith('.nii.gz')])
	df_list = []
	for idxLabelValue, labelValue in enumerate(label_values):
		mask_path = os.path.join(split_pred_nifti_path, label_names[idxLabelValue] + '.nii.gz')
		# Run the extractor
		print(f"ct nifti path : {ct_nifti_path}")
		print(f" mask path : {mask_path}")
		print(f"labelValue : {labelValue}")
		print(f"label_name : {str(label_names[idxLabelValue])}")
		# Instantiate the extractor and modify the settings to keep the 3D shape features
		extractor = featureextractor.RadiomicsFeatureExtractor(correctMask=True,
																													 geometryTolerance=float(1e-4),
																													 label=int(labelValue))
		extractor.settings['minimumROIDimensions'] = 2
		extractor.disableAllFeatures()
		extractor.enableFeaturesByName(shape=shape_features)
		result = extractor.execute(ct_nifti_path, mask_path) # dictionary
		# keep only the features we want
		# Get the corresponding label number -- all might not be present
		# corresponding_label_value = labelValue#label_values[label_names.index(label_names[n])]
		dict_keep = {'ReferencedSegment': int(labelValue),
								 'label_name': str(label_names[idxLabelValue])}
		keys_keep = [f for f in result.keys() if 'original_shape' in f]
		# Just keep the feature keys we want
		dict_keep_new_values = {key_keep: result[key_keep] for key_keep in keys_keep}
		dict_keep.update(dict_keep_new_values)
		df1 = pd.DataFrame([dict_keep])
		# change values of columns to remove original_shape_
		df1.columns = df1.columns.str.replace('original_shape_', '')
		# Append to the ReferencedSegment and label_name df
		df_list.append(df1)

	# concat all label features
	df = pd.concat(df_list)

	return df

In [51]:
def order_dicom_files_image_position(dcm_directory):
	"""
	Orders the dicom files according to image position and orientation.

	Arguments:
		dcm_directory : input directory of dcm files to put in order

	Outputs:
		files_sorted   : dcm files in sorted order
		sop_all_sorted : the SOPInstanceUIDs in sorted order
		pos_all_sorted : the image position in sorted order

	"""
	files = [os.path.join(dcm_directory,f) for f in os.listdir(dcm_directory)]

	num_files = len(files)

	pos_all = []
	sop_all = []

	for n in range(0,num_files):
		# read dcm file
		filename = files[n]
		ds = dcmread(filename)

		# get ImageOrientation (0020, 0037)
		# ImageOrientation = ds['0x0020','0x0037'].value
		ImageOrientation = ds.ImageOrientationPatient

		# get ImagePositionPatient (0020, 0032)
		# ImagePositionPatient = ds['0x0020','0x0032'].value
		ImagePositionPatient = ds.ImagePositionPatient

		# calculate z value
		x_vector = ImageOrientation[0:3]
		y_vector = ImageOrientation[3:]
		z_vector = np.cross(x_vector,y_vector)

		# multiple z_vector by ImagePositionPatient
		pos = np.dot(z_vector,ImagePositionPatient)
		pos_all.append(pos)

		# get the SOPInstanceUID
		# sop = ds['0x0008', '0x0018'].value
		sop = ds.SOPInstanceUID
		sop_all.append(sop)


	#----- order the SOPInstanceUID/files by z value ----#

	sorted_ind = np.argsort(pos_all)
	pos_all_sorted = np.array(pos_all)[sorted_ind.astype(int)]
	sop_all_sorted = np.array(sop_all)[sorted_ind.astype(int)]
	files_sorted = np.array(files)[sorted_ind.astype(int)]

	return files_sorted, sop_all_sorted, pos_all_sorted

In [52]:
def create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
																												 SOPInstanceUID_seg,
																												 seg_file,
																												 dcm_directory,
																												 segments_code_mapping_df,
																												 shape_features_code_mapping_df,
																												 df_features,
																												 SegmentAlgorithmName,
																												 SeriesDescriptionPrefix
																												 ):

	"""Function that creates the metajson necessary for the creation of a
	structured report from a pandas dataframe of label names and features for
	each.

	Inputs:
		SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
																			file
		SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
		seg_file                        : filename of SEG DCM file
		dcm_directory                   : ct directory that will be sorted in
																			terms of axial ordering according to the
																			ImagePositionPatient and ImageOrientation
																			fields
		segments_code_mapping_df        : dataframe that holds the names of the
																			segments and the associated code values etc.
		shape_features_code_mapping_df  : dataframe that holds the names of the
																			features and the associated code values etc.
		df_features                     : a pandas dataframe holding the segments and a
																			set of 3D shape features for each
		SegmentAlgorithmName            : the name of the algorithm used to create the
																			segmentations - e.g. '3d_fullres_tta_nnUnet'

	Outputs:
		Returns the metajson for the structured report that will then be used by
		dcmqi tid1500writer to create a structured report
	"""

	# --- Get the version number for the pyradiomics package --- #

	pyradiomics_version_number = str(radiomics.__version__)

	# --- Sort the dcm files first according to --- #
	# --- ImagePositionPatient and ImageOrientation --- #

	files_sorted, sop_all_sorted, pos_all_sorted = order_dicom_files_image_position(dcm_directory)
	files_sorted = [os.path.basename(f) for f in files_sorted]

	# --- Create the header for the json --- #

	inputMetadata = {}
	inputMetadata["@schema"]= "https://raw.githubusercontent.com/qiicr/dcmqi/master/doc/schemas/sr-tid1500-schema.json#"
	# inputMetadata["SeriesDescription"] = "Measurements"
	inputMetadata["SeriesDescription"] = SeriesDescriptionPrefix + '_' + "Measurements"
	inputMetadata["SeriesNumber"] = "1001"
	inputMetadata["InstanceNumber"] = "1"

	inputMetadata["compositeContext"] = [abspath(seg_file)] # not full path

	inputMetadata["imageLibrary"] = files_sorted # not full path

	# inputMetadata["observerContext"] = {
	#                                     "ObserverType": "PERSON",
	#                                     "PersonObserverName": "Reader1"
	#                                   }
	# inputMetadata["observerContext"] = {
	#                     "ObserverType": "DEVICE",
	#                     "DeviceObserverName": "pyradiomics",
	#                     "DeviceObserverModelName": "v3.0.1"
	#                   }
	inputMetadata["observerContext"] = {
											"ObserverType": "DEVICE",
											"DeviceObserverName": "pyradiomics",
											"DeviceObserverModelName": pyradiomics_version_number
										}

	inputMetadata["VerificationFlag"]  = "UNVERIFIED"
	inputMetadata["CompletionFlag"] =  "COMPLETE"
	inputMetadata["activitySession"] = "1"
	inputMetadata["timePoint"] = "1"

	# ------------------------------------------------------------------------- #
	# --- Create the measurement_dict for each segment - holds all features --- #

	measurement = []

	# --- Now create the dict for all features and all segments --- #

	# --- Loop over the number of segments --- #

	# number of rows in the df_features
	num_segments = df_features.shape[0]

	# Array of dictionaries - one dictionary for each segment
	measurement_across_segments_combined = []

	for segment_id in range(0,num_segments):

		ReferencedSegment = int(df_features['ReferencedSegment'].values[segment_id])
		FindingSite = df_features['label_name'].values[segment_id]

		print('segment_id: ' + str(segment_id))
		print('ReferencedSegment: ' + str(ReferencedSegment))
		print('FindingSite: ' + str(FindingSite))

		# --- Create the dict for the Measurements group --- #
		TrackingIdentifier = "Measurements group " + str(ReferencedSegment)

		segment_row = segments_code_mapping_df[segments_code_mapping_df["segment"] == FindingSite]
		# print(segment_row)

		my_dict = {
			"TrackingIdentifier": str(TrackingIdentifier),
			"ReferencedSegment": int(ReferencedSegment),
			"SourceSeriesForImageSegmentation": str(SeriesInstanceUID),
			"segmentationSOPInstanceUID": str(SOPInstanceUID_seg),
			"measurementAlgorithmIdentification": {
								"AlgorithmName": "pyradiomics",
								"AlgorithmVersion": str(pyradiomics_version_number)
						},
			"Finding": {
				"CodeValue": "113343008",
				"CodingSchemeDesignator": "SCT",
				"CodeMeaning": "Organ"
			},
			"FindingSite": {
				"CodeValue": str(segment_row["FindingSite_CodeValue"].values[0]),
				"CodingSchemeDesignator": str(segment_row["FindingSite_CodingSchemeDesignator"].values[0]),
				"CodeMeaning": str(segment_row["FindingSite_CodeMeaning"].values[0])
			}
		}

		measurement = []
		# number of features - number of columns in df_features - 2 (label_name and ReferencedSegment)
		num_values = len(df_features.columns)-2

		feature_list = df_features.columns[2:] # remove first two


		# For each measurement per region segment
		for n in range(0,num_values):
			measurement_dict = {}
			row = df_features.loc[df_features['label_name'] == FindingSite]
			feature_row = shape_features_code_mapping_df.loc[shape_features_code_mapping_df["shape_feature"] == feature_list[n]]
			value = str(np.round(row[feature_list[n]].values[0],3))
			measurement_dict["value"] = value
			measurement_dict["quantity"] = {}
			measurement_dict["quantity"]["CodeValue"] = str(feature_row["quantity_CodeValue"].values[0])
			measurement_dict["quantity"]["CodingSchemeDesignator"] = str(feature_row["quantity_CodingSchemeDesignator"].values[0])
			measurement_dict["quantity"]["CodeMeaning"] = str(feature_row["quantity_CodeMeaning"].values[0])
			measurement_dict["units"] = {}
			measurement_dict["units"]["CodeValue"] = str(feature_row["units_CodeValue"].values[0])
			measurement_dict["units"]["CodingSchemeDesignator"] = str(feature_row["units_CodingSchemeDesignator"].values[0])
			measurement_dict["units"]["CodeMeaning"] = str(feature_row["units_CodeMeaning"].values[0])
			# measurement_dict["measurementAlgorithmIdentification"] = {}
			# measurement_dict["measurementAlgorithmIdentification"]["AlgorithmName"] = "pyradiomics"
			# measurement_dict["measurementAlgorithmIdentification"]["AlgorithmVersion"] = str(pyradiomics_version_number)
			measurement.append(measurement_dict)

		measurement_combined_dict = {}
		measurement_combined_dict['measurementItems'] = measurement # measurement is an array of dictionaries

		output_dict_one_segment = {**my_dict, **measurement_combined_dict}

		# append to array for all segments

		measurement_across_segments_combined.append(output_dict_one_segment)

	# --- Add the measurement data --- #

	inputMetadata["Measurements"] = {}
	inputMetadata["Measurements"] = measurement_across_segments_combined

	return inputMetadata

In [53]:
def save_structured_report_for_shape_features_idc_seg(SeriesInstanceUID,
																							label_values,
																							label_names,
																							SOPInstanceUID_seg,
																							pred_dicomseg_path,
																							dicomseg_json_path,
																							dcm_directory,
																							split_pred_nii_path,
																							ct_nifti_path,
																							segments_code_mapping_df,
																							shape_features_code_mapping_df,
																							sr_json_path,
																							sr_path,
																							SegmentAlgorithmName,
																							SeriesDescriptionPrefix
																							):

	""" This function creates the SR necessary for the nnUNet shape features

	Inputs:
	SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
																		file
	SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
	pred_dicomseg_path              : filename of DICOM SEG file
	dicomseg_json_path              : json file for DICOM SEG file
	dcm_directory                   : list of ct files that will be sorted in
																		terms of axial ordering according to the
																		ImagePositionPatient and ImageOrientation
																		fields
	nnunet_base_path                : path to hold the split nifti files
	ct_nifti_path                   : filename for CT nifti file
	segments_code_mapping_df        : dataframe that holds the names of the
																		segments and the associated code values etc.
	shape_features_code_mapping_df  : dataframe that holds the names of the
																		features and the associated code values etc.
	sr_json_path                    : the path that the metajson for the SR for
																		the 3D shape features will be saved
	sr_path                         : the path that the SR for the 3D shape
																		features will be saved
	SegmentAlgorithmName            : the name of the algorithm used to create the
																		segmentations - e.g. '3d_fullres_tta_nnUnet'

	Outputs:
		Returns the metajson for the structured report that will then be used by
		dcmqi tid1500writer to create a structured report

	"""

	# --- get label values and names from metajson file --- #
	label_values, label_names = label_values, label_names#get_label_and_names_from_metadata_json(dicomseg_json_path)

	df_features = compute_pyradiomics_3D_features(ct_nifti_path,
																								label_values,
																								label_names,
																								split_pred_nii_path,
																								nnunet_shape_features_code_mapping_df)
	print ('created df_features')

	# --- upload csv file to bucket --- #
	# !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_features_csv_path $gs_uri_features_csv_file

	# remove nii files after saving out pyradiomics results
	!rm $split_pred_nii_path/*
	# remove csv
	# !rm $pred_features_csv_path

	# --- Create the final metadata for the SR --- #
	inputMetadata = create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
																																			 SOPInstanceUID_seg,
																																			 pred_dicomseg_path,
																																			 dcm_directory,
																																			 segments_code_mapping_df,
																																			 nnunet_shape_features_code_mapping_df,
																																			 df_features,
																																			 SegmentAlgorithmName,
																																			 SeriesDescriptionPrefix)

	print ('created SR json for shape features')

	# --- Write out json --- #

	with open(sr_json_path, 'w') as f:
		json.dump(inputMetadata, f, indent=2)
	print ('wrote out json for shape features')

	# --- Save the SR for nnUNet shape features --- #
	inputImageLibraryDirectory = dcm_directory
	outputDICOM = sr_path
	# the name of the folder where the seg files are located
	inputCompositeContextDirectory = os.path.basename(pred_dicomseg_path) # might need to check this
	inputMetadata_json = sr_json_path

	print ('inputImageLibraryDirectory: ' + str(inputImageLibraryDirectory))
	print ('outputDICOM: ' + str(outputDICOM))
	print ('inputCompositeContextDirectory: ' + str(inputCompositeContextDirectory))
	print ('inputMetadata_json: ' + str(inputMetadata_json))
	!tid1500writer --inputImageLibraryDirectory $inputImageLibraryDirectory \
								--outputDICOM $outputDICOM  \
								--inputCompositeContextDirectory $inputCompositeContextDirectory \
								--inputMetadata $inputMetadata_json
	print ('wrote out SR for shape features')

	return

# Imports

In [54]:
# Packages for the structured report
import radiomics
from radiomics import featureextractor

import highdicom

from pathlib import Path

import highdicom as hd

from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

from highdicom.sr.content import (
	FindingSite,
	ImageRegion,
	ImageRegion3D,
	SourceImageForRegion,
	SourceImageForMeasurement,
	SourceImageForMeasurementGroup
)
from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
	DeviceObserverIdentifyingAttributes,
	Measurement,
	MeasurementProperties,
	MeasurementReport,
	MeasurementsAndQualitativeEvaluations,
	ObservationContext,
	ObserverContext,
	PersonObserverIdentifyingAttributes,
	PlanarROIMeasurementsAndQualitativeEvaluations,
	RelationshipTypeValues,
	TrackingIdentifier,
	QualitativeEvaluation,
	ImageLibrary,
	ImageLibraryEntryDescriptors
)
from highdicom.sr.value_types import (
	CodedConcept,
	CodeContentItem,
)

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

# Input setup

In [ ]:
!lz4 -dc < $SEG_OUTPUT | tar xvf - -C $SEG_OUTPUT_UNZIP

In [ ]:
#nnunet code mapping
import pandas as pd
#radiomics df
!wget -N https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/nnunet_shape_features_code_mapping.csv
nnunet_shape_features_code_mapping_df = pd.read_csv("nnunet_shape_features_code_mapping.csv")

# Radiomics computation



In [57]:
seg_dicom_objects = glob.glob(os.path.join(SEG_OUTPUT_UNZIP, "**",
										 "*.dcm"), recursive=True)
assert os.path.exists(seg_dicom_objects[0])

In [ ]:
for serieUID_path in seg_dicom_objects:
	try:
		#convert AI SEG DICOM object to NIFTI SEG
		serieUID_image = str(pydicom.dcmread(serieUID_path).ReferencedSeriesSequence[0].SeriesInstanceUID)
		print(f"serieUID image referenced : {serieUID_image}")
		!mkdir -p {os.path.join(seg_nii_folder, serieUID_image)}
		assert os.path.exists(serieUID_path)
		!segimage2itkimage --inputDICOM $serieUID_path \
		--outputDirectory {os.path.join(seg_nii_folder, serieUID_image)} --outputType nii
		#download IDC reference image for downstream resampling
		out_image_dicom = os.path.join(IDC_IMAGE_DICOM, serieUID_image)
		!rm -rf $out_image_dicom
		!mkdir -p $out_image_dicom
		client.download_from_selection(seriesInstanceUID=list([serieUID_image]),
																								downloadDir=out_image_dicom,)
		#convert idc reference image to NIFTI/NRRD
		out_image_nii = os.path.join(IDC_IMAGE_NII, serieUID_image)
		out_image_nrrd = os.path.join(IDC_IMAGE_NRRD, serieUID_image)
		##nii
		!rm -rf $out_image_nii
		!mkdir -p $out_image_nii
		##nrrd
		!rm -rf $out_image_nrrd
		!mkdir -p $out_image_nrrd
		#convert image dicom to nii
		!dcm2niix -z y -m y -f %i_{serieUID_image} -o $out_image_nii $out_image_dicom
		#convert image dicom to nrrd
		out_image_dicom_for_nrrd = "/".join(glob.glob(os.path.join(out_image_dicom, "**", "*.dcm"), recursive=True)[0].split("/")[:-1])
		out_img_nrrd_file_path = os.path.join(out_image_nrrd, f"{serieUID_image}.nrrd")
		convert_nrrd_args = {"input":out_image_dicom_for_nrrd, "output-img":out_img_nrrd_file_path}
		pypla.convert(verbose=True, **convert_nrrd_args)
		ref_image_nii_path = glob.glob(os.path.join(out_image_nii, '*.nii.gz'))[0]
		ref_image_nrrd_path = glob.glob(os.path.join(out_image_nrrd, '*.nrrd'))[0]
		#resample AI NIFTI SEG object to the reference image space
		output_serie_resampled = os.path.join(seg_nii_resampled_folder, serieUID_image)
		!rm -rf $output_serie_resampled
		!mkdir -p $output_serie_resampled
		for ai_segment_path in glob.glob(os.path.join(seg_nii_folder, serieUID_image, "*.nii.gz")):
			print(ai_segment_path)
			out_segment_path = os.path.join(output_serie_resampled, ai_segment_path.split('/')[-1])
			# !plastimatch resample --input $ai_segment_path \
			# --fixed $ref_image_nii_path --output $out_segment_path --interpolation nn
			resample_args = {"input" : ai_segment_path,#change to pred path if no largest_component_retrieval necessary
														"output" : out_segment_path,
														"fixed" : ref_image_nrrd_path,#ref_image_nii_path,
														"interpolation" : "nn"}
			pypla.resample(verbose = False, **resample_args)
		#compute radiomics
		##get parameters for radiomics computation
		SeriesInstanceUID = serieUID_image # image reference serieUID
		SOPInstanceUID = get_tag_pydicom(serieUID_path, "SOPInstanceUID").value #SOPInstanceUID of AI DICOM SEG
		ct_nifti_path = abspath(glob.glob(os.path.join(out_image_nrrd, "*.nrrd"))[0])#abspath(glob.glob(f"{out_image_nii}/*.nii.gz")[0])
		ct_nifti_path = abspath(ct_nifti_path)
		pred_dicomseg_path = serieUID_path
		dicomseg_json_path = abspath(glob.glob(os.path.join(seg_nii_folder, serieUID_image, "meta.json"))[0])
		mhub_ai_meta_df = retrieve_meta_df_from_json(dicomseg_json_path)
		dcm_directory = abspath("/".join(glob.glob(os.path.join(out_image_dicom, "**/*.dcm"), recursive=True)[0].split("/")[:-1]))
		segments_code_mapping_df = mhub_ai_meta_df#idc_seg_segments_code_mapping_df
		shape_features_code_mapping_df = nnunet_shape_features_code_mapping_df
		mhub_serieUID_sr_path = abspath(os.path.join(seg_sr_dicom_folder, serieUID_image))
		mhub_serieUID_sr_json_path = abspath(os.path.join(seg_sr_json_dicom_folder, serieUID_image))
		!mkdir -p $mhub_serieUID_sr_path
		!mkdir -p $mhub_serieUID_sr_json_path
		sr_json_path = abspath(os.path.join(mhub_serieUID_sr_json_path, serieUID_image+"_SR.json"))
		sr_path = abspath(os.path.join(mhub_serieUID_sr_path, serieUID_image+"_SR.dcm"))
		if "SegmentAlgorithmName" in list(mhub_ai_meta_df.columns.values):
			SegmentAlgorithmName = str(mhub_ai_meta_df.SegmentAlgorithmName.unique()[0])
		else:#if SegmentAlgorithmName not in dcmqi.meta.json, that means its likely that it is an expert annotation
			SegmentAlgorithmName = "MANUAL"
		print(f"SegmentAlgorithmName : {SegmentAlgorithmName}")
		segseriesDescription = ""
		if len(terraRadSeriesDescription) > 0:
			segseriesDescription = terraRadSeriesDescription
		else:
			segseriesDescription = SegmentAlgorithmName
		label_values_lst = []
		label_names_lst = []
		for analyzed_segmentCodeValue,analyzed_segmentCodingSchemeDesignator \
		in zip(dicom_sr_CodeValues_lst, dicom_sr_CodingSchemeDesignator_lst):
			if analyzed_segmentCodeValue in mhub_ai_meta_df.FindingSite_CodeValue.values \
			and analyzed_segmentCodingSchemeDesignator in mhub_ai_meta_df.FindingSite_CodingSchemeDesignator.values:
				label_values_lst.append(int(mhub_ai_meta_df[(mhub_ai_meta_df.FindingSite_CodeValue \
																										== str(analyzed_segmentCodeValue))\
																										&(mhub_ai_meta_df.FindingSite_CodingSchemeDesignator\
																											== str(analyzed_segmentCodingSchemeDesignator))].labelID.unique()[0]))
				label_names_lst.append(str(mhub_ai_meta_df[(mhub_ai_meta_df.FindingSite_CodeValue \
																									== str(analyzed_segmentCodeValue))\
																									&(mhub_ai_meta_df.FindingSite_CodingSchemeDesignator\
																											== str(analyzed_segmentCodingSchemeDesignator))].FindingSite_CodeMeaning.unique()[0]))
			label_values = label_values_lst
			label_names = label_names_lst
		print(f"label_values: {label_values}")
		print(f"label_names: {label_names}")
		#rename dcmqi seg output by segmentCodeMeaning for SR computation
		out_serieUID_renamed_path = os.path.join(seg_nii_resampled_renamed_folder, serieUID_image)
		!rm -rf $out_serieUID_renamed_path && mkdir -p $out_serieUID_renamed_path
		for segment_dcmqi_path in glob.glob(f"{output_serie_resampled}/*nii.gz"):
			segment_name = str(mhub_ai_meta_df[mhub_ai_meta_df.labelID == \
																		int(segment_dcmqi_path.split("/")[-1][:-7])].FindingSite_CodeMeaning.unique()[0])
			shutil.copy(segment_dcmqi_path, os.path.join(out_serieUID_renamed_path, f"{segment_name}.nii.gz"))
		split_pred_nii_path = abspath(out_serieUID_renamed_path)
		nnunet_features_metajson = save_structured_report_for_shape_features_idc_seg(SeriesInstanceUID=SeriesInstanceUID,
																																			SOPInstanceUID_seg=SOPInstanceUID,
																																			label_values=label_values,
																																			label_names = label_names,
																																			pred_dicomseg_path=pred_dicomseg_path,
																																			dicomseg_json_path=dicomseg_json_path,
																																			dcm_directory=dcm_directory,
																																			split_pred_nii_path=split_pred_nii_path,
																																			ct_nifti_path=ct_nifti_path,
																																			segments_code_mapping_df=segments_code_mapping_df,
																																			shape_features_code_mapping_df=shape_features_code_mapping_df,
																																			sr_json_path=sr_json_path,
																																			sr_path=sr_path,
																																			SegmentAlgorithmName=SegmentAlgorithmName,
																																			SeriesDescriptionPrefix=segseriesDescription)
		print("\n")
	except:
		print("DID NOT WORK FOR SERIEUID : {serieUID_image}")